In [1]:
from pathlib import Path
import pandas as pd
#import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gzip
import pickle
import json
#import binpickle
#from natural.size import binarysize

In [4]:
topics = pd.read_json('trec_topics.json.gz', lines = True)
topics

ValueError: Trailing data

In [3]:
meta = pd.read_json('data/trec_metadata.json.gz', lines = True)
meta

,page_id,quality_score,quality_score_disc,geographic_locations
0,12,0.909000,FA,[]
1,25,0.909000,FA,[]
2,39,0.750164,GA,[]
3,290,0.560753,B,[]
4,303,0.909000,FA,[Northern America]
...,...,...,...,...
6023423,67268654,0.126549,Stub,[]
6023424,67268663,0.194086,Start,[Europe]
6023425,67268668,0.424822,C,[]
6023426,67268699,0.369754,C,[]


In [26]:
def seed_rel_docs(row, meta):
    df = meta[meta['page_id'].isin(row)][['page_id', 'quality_score']]
    df = df.sort_values(by='quality_score', ascending = False)
    #print(df.head(5))
    return df['page_id'].values

In [27]:
topics['sorted_articles'] = topics['rel_docs'].apply(lambda x: seed_rel_docs(x, meta))

In [28]:
topics['seed_articles'] = topics['sorted_articles'].apply(lambda x: x[:10])
topics

,id,title,keywords,scope,homepage,rel_docs,sorted_articles,seed_articles
0,1,Agriculture,"[agriculture, crops, livestock, forests, farming]",This WikiProject strives to develop and improv...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[572, 627, 903, 1193, 1542, 1634, 3751, 3866, ...","[45341506, 54101623, 31728658, 38121957, 40838...","[45341506, 54101623, 31728658, 38121957, 40838..."
1,2,Architecture,"[architecture, skyscraper, landscape, building...",This WikiProject aims to: 1. Thoroughly explor...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[682, 954, 1170, 1315, 1322, 1324, 1325, 1435,...","[2407040, 52682014, 6531850, 3978080, 59495810...","[2407040, 52682014, 6531850, 3978080, 59495810..."
2,3,Athletics,"[athletics, player, sports, game, gymnastics]","WikiProject Athletics, a project focused on im...",https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[5729, 8490, 9623, 10391, 12231, 13791, 16078,...","[40882539, 57032900, 1571218, 182265, 25725265...","[40882539, 57032900, 1571218, 182265, 25725265..."
3,4,Aviation,"[aviation, aircraft, airplane, airship, pilot,...",The project generally considers any article re...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[849, 852, 1293, 1902, 1942, 2039, 2075, 2082,...","[75082, 565977, 44111577, 2313495, 43543066, 4...","[75082, 565977, 44111577, 2313495, 43543066, 4..."
4,5,Baseball,[baseball],Articles pertaining to baseball including base...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[1135, 1136, 1293, 1893, 2129, 2140, 3797, 380...","[43784975, 54755172, 47714172, 27672272, 47762...","[43784975, 54755172, 47714172, 27672272, 47762..."
5,6,Biography/science and academia work group,"[academia biography, academia, scientists, sch...",The Science and Academia Work Group is a worki...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[308, 340, 700, 728, 736, 784, 851, 852, 892, ...","[4096055, 11353703, 14540397, 2210754, 1529557...","[4096055, 11353703, 14540397, 2210754, 1529557..."
6,7,Biography/sports and games work group,"[sports biography, athlets, player, memoir, re...",The Sports and Games Work Group is a working g...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[595, 890, 1020, 1135, 1293, 1806, 1893, 2575,...","[5111581, 1532004, 41976318, 35040446, 1996967...","[5111581, 1532004, 41976318, 35040446, 1996967..."
7,8,Biography/WikiProject Actors and Filmmakers,"[actors, filmmaker, actress, film director, pe...",WikiProject Actors and Filmmakers covers biogr...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[344, 676, 808, 872, 1247, 1806, 1828, 2083, 2...","[6744230, 5684890, 4377306, 47786254, 4253063,...","[6744230, 5684890, 4377306, 47786254, 4253063,..."
8,9,Boxing,[boxing],This Wikiproject aims to describe a template f...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[4243, 17163, 25506, 39027, 45360, 45772, 4811...","[30769008, 985206, 17994055, 36520532, 39027, ...","[30769008, 985206, 17994055, 36520532, 39027, ..."
9,10,Buddhism,"[buddhism, buddha, buddhist, buddhist temple, ...",WikiProject Buddhism is a group of people dedi...,https://en.wikipedia.org/wiki/Wikipedia:WikiPr...,"[1017, 1178, 1239, 1669, 1735, 2642, 2784, 284...","[2184535, 14995516, 29399372, 456706, 323186, ...","[2184535, 14995516, 29399372, 456706, 323186, ..."


In [29]:
topic_info = topics[['id', 'keywords', 'title', 'seed_articles']]

In [30]:
topic_info = topic_info.explode('seed_articles')
topic_info.rename(columns={'seed_articles':'page_id'},inplace=True)

In [31]:
topic_info

,id,keywords,title,page_id
0,1,"[agriculture, crops, livestock, forests, farming]",Agriculture,45341506
0,1,"[agriculture, crops, livestock, forests, farming]",Agriculture,54101623
0,1,"[agriculture, crops, livestock, forests, farming]",Agriculture,31728658
0,1,"[agriculture, crops, livestock, forests, farming]",Agriculture,38121957
0,1,"[agriculture, crops, livestock, forests, farming]",Agriculture,40838871
...,...,...,...,...
56,57,"[nigeria, nigerians, nigerian languages, niger...",Nigeria,61584270
56,57,"[nigeria, nigerians, nigerian languages, niger...",Nigeria,24130928
56,57,"[nigeria, nigerians, nigerian languages, niger...",Nigeria,7406065
56,57,"[nigeria, nigerians, nigerian languages, niger...",Nigeria,56194341


In [32]:
topic_info.to_parquet('data/topic_info.parquet')

In [65]:
relevant_docs = set()
for articles in topic_info['seed_articles']:
    for item in articles:
        relevant_docs.add(item)

In [67]:
relevant_docs = list(relevant_docs)

In [68]:
data = []
file = 'data/trec_corpus.json.gz'
#for file in glob.glob('logs/*.json'):
with open(file, 'rb') as fd:
    f = gzip.GzipFile(fileobj=fd)
    for line in f:
        if json.loads(line)['id'] in relevant_docs:
            data.append(json.loads(line))

pages = pd.DataFrame(data)

In [75]:
pages.to_parquet('data/selected_pages.parquet')

In [5]:
pages = pd.read_parquet('data/selected_pages.parquet')

In [66]:
pages.head()

,page_id,title,text,url,cleaned_text,keywords
0,5399,Colorado,{{short description|State of the United States...,https://en.wikipedia.org/wiki/Colorado,"Colorado (, other variants) is a state in the ...","[('colorado', 0.574), ('coloradoan', 0.441), (..."
1,5766,Clement Attlee,{{Redirect|Attlee}}\n{{short description|Prime...,https://en.wikipedia.org/wiki/Clement_Attlee,"Clement Richard Attlee, 1st Earl Attlee, (3 Ja...","[('churchill', 0.4173), ('thatcher', 0.3769), ..."
2,6749,Cheerleading,{{about|American-style cheerleading|Japanese-s...,https://en.wikipedia.org/wiki/Cheerleading,Cheerleading is an activity in which the parti...,"[('cheerleading', 0.6736), ('cheerleader', 0.5..."
3,11668,Follies,{{Other uses|Follies (disambiguation)}}\n{{Inf...,https://en.wikipedia.org/wiki/Follies,Follies is a musical with music and lyrics by ...,"[('broadway', 0.4292), ('broadwayfollies', 0.4..."
4,15080,Indian removal,{{Short description|Early 19th-century United ...,https://en.wikipedia.org/wiki/Indian_removal,Indian removal is the former United States gov...,"[('iroquois', 0.3307), ('cherokee', 0.313), ('..."


In [70]:
#pages = pd.read_json('data/trec_corpus.json.gz', lines = True, nrows=100)
#pages = pages.drop_duplicates('page_id')
#pages

In [6]:
MEDIA_PREFIXES = ['File', 'Image', 'Media']
CAT_PREFIXES = ['Category']

MEDIA_ALIASES = {
    "ab": ["Медиа", "Файл", "Афаил", "Амедиа", "Изображение"],
    "ace": ["Beureukaih", "Gambar", "Alat", "Berkas"],
    "ady": ["Медиа"],
    "af": ["Lêer", "Beeld"],
    "als": ["Medium", "Datei", "Bild"],
    "am": ["ፋይል", "ስዕል"],
    "an": ["Imachen", "Imagen"],
    "ang": ["Ymele", "Biliþ"],
    "ar": ["ميديا", "صورة", "وسائط", "ملف"],
    "arc": ["ܠܦܦܐ", "ܡܝܕܝܐ"],
    "arz": ["ميديا", "صورة", "وسائط", "ملف"],
    "as": ["চিত্ৰ", "चित्र", "চিত্র", "মাধ্যম"],
    "ast": ["Imaxen", "Ficheru", "Imaxe", "Archivu", "Imagen", "Medios"],
    "atj": ["Tipatcimoctakewin", "Natisinahikaniwoc"],
    "av": ["Медиа", "Файл", "Изображение"],
    "ay": ["Medio", "Archivo", "Imagen"],
    "az": ["Mediya", "Şəkil", "Fayl"],
    "azb": ["رسانه", "تصویر", "مدیا", "فایل", "رسانه‌ای"],
    "ba": ["Медиа", "Рәсем", "Файл", "Изображение"],
    "bar": ["Medium", "Datei", "Bild"],
    "bat-smg": ["Vaizdas", "Medėjė", "Abruozdielis"],
    "bcl": ["Medio", "Ladawan"],
    "be": ["Мультымедыя", "Файл", "Выява"],
    "be-x-old": ["Мэдыя", "Файл", "Выява"],
    "bg": ["Медия", "Файл", "Картинка"],
    "bh": ["मीडिया", "चित्र"],
    "bjn": ["Barakas", "Gambar", "Berkas"],
    "bm": ["Média", "Fichier"],
    "bn": ["চিত্র", "মিডিয়া"],
    "bpy": ["ছবি", "মিডিয়া"],
    "br": ["Skeudenn", "Restr"],
    "bs": ["Mediji", "Slika", "Datoteka", "Medija"],
    "bug": ["Gambar", "Berkas"],
    "bxr": ["Файл", "Меди", "Изображение"],
    "ca": ["Fitxer", "Imatge"],
    "cbk-zam": ["Medio", "Archivo", "Imagen"],
    "cdo": ["文件", "媒體", "圖像", "檔案"],
    "ce": ["Хlум", "Медиа", "Сурт", "Файл", "Медйа", "Изображение"],
    "ceb": ["Payl", "Medya", "Imahen"],
    "ch": ["Litratu"],
    "ckb": ["میدیا", "پەڕگە"],
    "co": ["Immagine"],
    "crh": ["Медиа", "Resim", "Файл", "Fayl", "Ресим"],
    "cs": ["Soubor", "Média", "Obrázok"],
    "csb": ["Òbrôzk", "Grafika"],
    "cu": ["Видъ", "Ви́дъ", "Дѣло", "Срѣдьства"],
    "cv": ["Медиа", "Ӳкерчĕк", "Изображение"],
    "cy": ["Delwedd"],
    "da": ["Billede", "Fil"],
    "de": ["Medium", "Datei", "Bild"],
    "din": ["Ciɛl", "Apamduööt"],
    "diq": ["Medya", "Dosya"],
    "dsb": ["Wobraz", "Dataja", "Bild", "Medija"],
    "dty": ["चित्र", "मिडिया"],
    "dv": ["ފައިލު", "މީޑިއާ", "ފައިލް"],
    "el": ["Εικόνα", "Αρχείο", "Μέσο", "Μέσον"],
    "eml": ["Immagine"],
    "eo": ["Dosiero", "Aŭdvidaĵo"],
    "es": ["Medio", "Archivo", "Imagen"],
    "et": ["Pilt", "Fail", "Meedia"],
    "eu": ["Irudi", "Fitxategi"],
    "ext": ["Archivu", "Imagen", "Mediu"],
    "fa": ["رسانه", "تصویر", "مدیا", "پرونده", "رسانه‌ای"],
    "ff": ["Média", "Fichier"],
    "fi": ["Kuva", "Tiedosto"],
    "fiu-vro": ["Pilt", "Meediä"],
    "fo": ["Miðil", "Mynd"],
    "fr": ["Média", "Fichier"],
    "frp": ["Émâge", "Fichiér", "Mèdia"],
    "frr": ["Medium", "Datei", "Bild"],
    "fur": ["Immagine", "Figure"],
    "fy": ["Ofbyld"],
    "ga": ["Íomhá", "Meán"],
    "gag": ["Mediya", "Medya", "Resim", "Dosya", "Dosye"],
    "gan": ["媒体文件", "文件", "文檔", "档案", "媒體", "图像", "圖像", "媒体", "檔案"],
    "gd": ["Faidhle", "Meadhan"],
    "gl": ["Imaxe", "Ficheiro", "Arquivo", "Imagem"],
    "glk": ["رسانه", "تصویر", "پرونده", "فاىل", "رسانه‌ای", "مديا"],
    "gn": ["Medio", "Imagen", "Ta'ãnga"],
    "gom": ["माध्यम", "मिडिया", "फायल"],
    "gor": ["Gambar", "Berkas"],
    "got": ["𐍆𐌴𐌹𐌻𐌰"],
    "gu": ["દ્રશ્ય-શ્રાવ્ય (મિડિયા)", "દ્રશ્ય-શ્રાવ્ય_(મિડિયા)", "ચિત્ર"],
    "gv": ["Coadan", "Meanyn"],
    "hak": ["文件", "媒體", "圖像", "檔案"],
    "haw": ["Kiʻi", "Waihona", "Pāpaho"],
    "he": ["תמונה", "קו", "מדיה", "קובץ"],
    "hi": ["मीडिया", "चित्र"],
    "hif": ["file", "saadhan"],
    "hr": ["Mediji", "DT", "Slika", "F", "Datoteka"],
    "hsb": ["Wobraz", "Dataja", "Bild"],
    "ht": ["Imaj", "Fichye", "Medya"],
    "hu": ["Kép", "Fájl", "Média"],
    "hy": ["Պատկեր", "Մեդիա"],
    "ia": ["Imagine", "Multimedia"],
    "id": ["Gambar", "Berkas"],
    "ig": ["Nká", "Midia", "Usòrò", "Ákwúkwó orünotu", "Ákwúkwó_orünotu"],
    "ii": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "ilo": ["Midia", "Papeles"],
    "inh": ["Медиа", "Файл", "Изображение"],
    "io": ["Imajo", "Arkivo"],
    "is": ["Miðill", "Mynd"],
    "it": ["Immagine"],
    "ja": ["メディア", "ファイル", "画像"],
    "jbo": ["velsku", "datnyvei"],
    "jv": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "ka": ["მედია", "სურათი", "ფაილი"],
    "kaa": ["Swret", "Таспа", "سۋرەت", "Taspa", "Su'wret", "Сурет", "تاسپا"],
    "kab": ["Tugna"],
    "kbd": ["Медиа", "Файл"],
    "kbp": ["Média", "Fichier"],
    "kg": ["Fisye"],
    "kk": ["Swret", "سۋرەت", "Таспа", "Taspa", "Сурет", "تاسپا"],
    "kl": ["Billede", "Fiileq", "Fil"],
    "km": ["ឯកសារ", "រូបភាព", "មេឌា", "មីឌា"],
    "kn": ["ಚಿತ್ರ", "ಮೀಡಿಯ"],
    "ko": ["미디어", "파일", "그림"],
    "koi": ["Медиа", "Файл", "Изображение"],
    "krc": ["Медиа", "Файл", "Изображение"],
    "ks": ["میڈیا", "فَیِل"],
    "ksh": ["Beld", "Meedije", "Medie", "Belld", "Medium", "Datei", "Meedijum", "Bild"],
    "ku": ["میدیا", "پەڕگە", "Medya", "Wêne"],
    "kv": ["Медиа", "Файл", "Изображение"],
    "kw": ["Restren"],
    "ky": ["Медиа", "Файл"],
    "la": ["Imago", "Fasciculus"],
    "lad": ["Dossia", "Medya", "Archivo", "Dosya", "Imagen", "Meddia"],
    "lb": ["Fichier", "Bild"],
    "lbe": ["Медиа", "Сурат", "Изображение"],
    "lez": ["Медиа", "Mediya", "Файл", "Şəkil", "Изображение"],
    "lfn": ["Fix"],
    "li": ["Afbeelding", "Plaetje", "Aafbeilding"],
    "lij": ["Immaggine", "Immagine"],
    "lmo": ["Immagine", "Imàjine", "Archivi"],
    "ln": ["Média", "Fichier"],
    "lo": ["ສື່ອ", "ສື່", "ຮູບ"],
    "lrc": ["رسانه", "تصویر", "رسانه‌ای", "جانیا", "أسگ", "ڤارئسگأر"],
    "lt": ["Vaizdas", "Medija"],
    "ltg": ["Medeja", "Fails"],
    "lv": ["Attēls"],
    "mai": ["मेडिया", "फाइल"],
    "map-bms": ["Barkas", "Medhia", "Gambar", "Médhia"],
    "mdf": ["Медиа", "Няйф", "Изображение"],
    "mg": ["Rakitra", "Sary", "Média"],
    "mhr": ["Медиа", "Файл", "Изображение"],
    "min": ["Gambar", "Berkas"],
    "mk": ["Податотека", "Медија", "Медиум", "Слика"],
    "ml": ["പ്രമാണം", "ചി", "മീഡിയ", "പ്ര", "ചിത്രം"],
    "mn": ["Медиа", "Файл", "Зураг"],
    "mr": ["चित्र", "मिडिया"],
    "mrj": ["Медиа", "Файл", "Изображение"],
    "ms": ["Fail", "Imej"],
    "mt": ["Midja", "Medja", "Stampa"],
    "mwl": ["Multimédia", "Fexeiro", "Ficheiro", "Arquivo", "Imagem"],
    "my": ["ဖိုင်", "မီဒီယာ"],
    "myv": ["Медия", "Артовкс", "Изображение"],
    "mzn": ["رسانه", "تصویر", "مه‌دیا", "مدیا", "پرونده", "رسانه‌ای"],
    "nah": ["Mēdiatl", "Īxiptli", "Imagen"],
    "nap": ["Fiùra", "Immagine"],
    "nds": ["Datei", "Bild"],
    "nds-nl": ["Ofbeelding", "Afbeelding", "Bestaand"],
    "ne": ["मीडिया", "चित्र"],
    "new": ["किपा", "माध्यम"],
    "nl": ["Bestand", "Afbeelding"],
    "nn": ["Fil", "Bilde", "Filpeikar"],
    "no": ["Fil", "Medium", "Bilde"],
    "nov": [],
    "nrm": ["Média", "Fichier"],
    "nso": ["Seswantšho"],
    "nv": ["Eʼelyaaígíí"],
    "oc": ["Imatge", "Fichièr", "Mèdia"],
    "olo": ["Kuva", "Medii", "Failu"],
    "or": ["ମାଧ୍ୟମ", "ଫାଇଲ"],
    "os": ["Ныв", "Медиа", "Файл", "Изображение"],
    "pa": ["ਤਸਵੀਰ", "ਮੀਡੀਆ"],
    "pcd": ["Média", "Fichier"],
    "pdc": ["Medium", "Datei", "Bild", "Feil"],
    "pfl": ["Dadai", "Medium", "Datei", "Bild"],
    "pi": ["मीडिया", "पटिमा"],
    "pl": ["Plik", "Grafika"],
    "pms": ["Figura", "Immagine"],
    "pnb": ["میڈیا", "تصویر", "فائل"],
    "pnt": ["Εικόνα", "Αρχείον", "Εικόναν", "Μέσον"],
    "ps": ["انځور", "رسنۍ", "دوتنه"],
    "pt": ["Multimédia", "Ficheiro", "Arquivo", "Imagem"],
    "qu": ["Midya", "Imagen", "Rikcha"],
    "rm": ["Multimedia", "Datoteca"],
    "rmy": ["Fişier", "Mediya", "Chitro", "Imagine"],
    "ro": ["Fişier", "Imagine", "Fișier"],
    "roa-rup": ["Fişier", "Imagine", "Fișier"],
    "roa-tara": ["Immagine"],
    "ru": ["Медиа", "Файл", "Изображение"],
    "rue": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "rw": ["Dosiye", "Itangazamakuru"],
    "sa": ["चित्रम्", "माध्यमम्", "सञ्चिका", "माध्यम", "चित्रं"],
    "sah": ["Миэдьийэ", "Ойуу", "Билэ", "Изображение"],
    "sat": ["ᱨᱮᱫ", "ᱢᱤᱰᱤᱭᱟ"],
    "sc": ["Immàgini"],
    "scn": ["Immagine", "Mmàggini", "Mèdia"],
    "sd": ["عڪس", "ذريعات", "فائل"],
    "se": ["Fiila"],
    "sg": ["Média", "Fichier"],
    "sh": ["Mediji", "Slika", "Медија", "Datoteka", "Medija", "Слика"],
    "si": ["රූපය", "මාධ්‍යය", "ගොනුව"],
    "sk": ["Súbor", "Obrázok", "Médiá"],
    "sl": ["Slika", "Datoteka"],
    "sq": ["Figura", "Skeda"],
    "sr": ["Датотека", "Medij", "Slika", "Медија", "Datoteka", "Медиј", "Medija", "Слика"],
    "srn": ["Afbeelding", "Gefre"],
    "stq": ["Bielde", "Bild"],
    "su": ["Média", "Gambar"],
    "sv": ["Fil", "Bild"],
    "sw": ["Faili", "Picha"],
    "szl": ["Plik", "Grafika"],
    "ta": ["படிமம்", "ஊடகம்"],
    "tcy": ["ಮಾದ್ಯಮೊ", "ಫೈಲ್"],
    "te": ["ఫైలు", "దస్త్రం", "బొమ్మ", "మీడియా"],
    "tet": ["Imajen", "Arquivo", "Imagem"],
    "tg": ["Акс", "Медиа"],
    "th": ["ไฟล์", "สื่อ", "ภาพ"],
    "ti": ["ፋይል", "ሜድያ"],
    "tk": ["Faýl"],
    "tl": ["Midya", "Talaksan"],
    "tpi": ["Fail"],
    "tr": ["Medya", "Resim", "Dosya", "Ortam"],
    "tt": ["Медиа", "Рәсем", "Файл", "Räsem", "Изображение"],
    "ty": ["Média", "Fichier"],
    "tyv": ["Медиа", "Файл", "Изображение"],
    "udm": ["Медиа", "Файл", "Суред", "Изображение"],
    "ug": ["ۋاسىتە", "ھۆججەت"],
    "uk": ["Медіа", "Медиа", "Файл", "Изображение", "Зображення"],
    "ur": ["میڈیا", "تصویر", "وسیط", "زریعہ", "فائل", "ملف"],
    "uz": ["Mediya", "Tasvir", "Fayl"],
    "vec": ["Immagine", "Imàjine", "Mèdia"],
    "vep": ["Pilt", "Fail"],
    "vi": ["Phương_tiện", "Tập_tin", "Hình", "Tập tin", "Phương tiện"],
    "vls": ["Afbeelding", "Ofbeeldienge"],
    "vo": ["Ragiv", "Magod", "Nünamakanäd"],
    "wa": ["Imådje"],
    "war": ["Medya", "Fayl", "Paypay"],
    "wo": ["Xibaarukaay", "Dencukaay"],
    "wuu": ["文件", "档案", "图像", "媒体"],
    "xal": ["Аһар", "Боомг", "Изображение", "Зург"],
    "xmf": ["მედია", "სურათი", "ფაილი"],
    "yi": ["מעדיע", "תמונה", "טעקע", "בילד"],
    "yo": ["Fáìlì", "Amóhùnmáwòrán", "Àwòrán"],
    "za": ["媒体文件", "文件", "档案", "图像", "媒体"],
    "zea": ["Afbeelding", "Plaetje"],
    "zh": ["媒体文件", "F", "文件", "媒體", "档案", "图像", "圖像", "媒体", "檔案"],
    "zh-classical": ["文件", "媒體", "圖像", "檔案"],
    "zh-min-nan": ["tóng-àn", "文件", "媒體", "Mûi-thé", "圖像", "檔案"],
    "zh-yue": ["檔", "档", "文件", "图", "媒體", "圖", "档案", "图像", "圖像", "媒体", "檔案"],
}

CAT_ALIASES = {
    "ab": ["Категория", "Акатегориа"],
    "ace": ["Kawan", "Kategori"],
    "af": ["Kategorie"],
    "ak": ["Nkyekyem"],
    "als": ["Kategorie"],
    "am": ["መደብ"],
    "an": ["Categoría"],
    "ang": ["Flocc"],
    "ar": ["تصنيف"],
    "arc": ["ܣܕܪܐ"],
    "arz": ["تصنيف"],
    "as": ["CAT", "শ্ৰেণী", "श्रेणी", "শ্রেণী"],
    "ast": ["Categoría"],
    "atj": ["Tipanictawin"],
    "av": ["Категория"],
    "ay": ["Categoría"],
    "az": ["Kateqoriya"],
    "azb": ["بؤلمه"],
    "ba": ["Төркөм", "Категория"],
    "bar": ["Kategorie"],
    "bat-smg": ["Kategorija", "Kateguorėjė"],
    "bcl": ["Kategorya"],
    "be": ["Катэгорыя"],
    "be-x-old": ["Катэгорыя"],
    "bg": ["Категория"],
    "bh": ["श्रेणी"],
    "bjn": ["Tumbung", "Kategori"],
    "bm": ["Catégorie"],
    "bn": ["বিষয়শ্রেণী", "വിഭാഗം"],
    "bpy": ["থাক"],
    "br": ["Rummad"],
    "bs": ["Kategorija"],
    "bug": ["Kategori"],
    "bxr": ["Категори", "Категория"],
    "ca": ["Categoria"],
    "cbk-zam": ["Categoría"],
    "cdo": ["分類"],
    "ce": ["Категори", "Тоба", "Кадегар"],
    "ceb": ["Kategoriya"],
    "ch": ["Katigoria"],
    "ckb": ["پ", "پۆل"],
    "co": ["Categoria"],
    "crh": ["Категория", "Kategoriya"],
    "cs": ["Kategorie"],
    "csb": ["Kategòrëjô"],
    "cu": ["Катигорї", "Категория", "Катигорїꙗ"],
    "cv": ["Категори"],
    "cy": ["Categori"],
    "da": ["Kategori"],
    "de": ["Kategorie"],
    "din": ["Bekätakthook"],
    "diq": ["Kategoriye", "Kategori"],
    "dsb": ["Kategorija"],
    "dty": ["श्रेणी"],
    "dv": ["ޤިސްމު"],
    "el": ["Κατηγορία"],
    "eml": ["Categoria"],
    "eo": ["Kategorio"],
    "es": ["CAT", "Categoría"],
    "et": ["Kategooria"],
    "eu": ["Kategoria"],
    "ext": ["Categoría", "Categoria"],
    "fa": ["رده"],
    "ff": ["Catégorie"],
    "fi": ["Luokka"],
    "fiu-vro": ["Katõgooria"],
    "fo": ["Bólkur"],
    "fr": ["Catégorie"],
    "frp": ["Catègorie"],
    "frr": ["Kategorie"],
    "fur": ["Categorie"],
    "fy": ["Kategory"],
    "ga": ["Rang", "Catagóir"],
    "gag": ["Kategori", "Kategoriya"],
    "gan": ["分類", "分类"],
    "gd": ["Roinn-seòrsa"],
    "gl": ["Categoría"],
    "glk": ["جرگه", "رده"],
    "gn": ["Ñemohenda"],
    "gom": ["वर्ग", "श्रेणी"],
    "gor": ["Dalala"],
    "got": ["𐌷𐌰𐌽𐍃𐌰"],
    "gu": ["શ્રેણી", "CAT", "શ્રે"],
    "gv": ["Ronney"],
    "hak": ["分類"],
    "haw": ["Māhele"],
    "he": ["קטגוריה", "קט"],
    "hi": ["श्र", "श्रेणी"],
    "hif": ["vibhag"],
    "hr": ["CT", "KT", "Kategorija"],
    "hsb": ["Kategorija"],
    "ht": ["Kategori"],
    "hu": ["Kategória"],
    "hy": ["Կատեգորիա"],
    "ia": ["Categoria"],
    "id": ["Kategori"],
    "ie": ["Categorie"],
    "ig": ["Ébéonọr", "Òtù"],
    "ii": ["分类"],
    "ilo": ["Kategoria"],
    "inh": ["ОагӀат"],
    "io": ["Kategorio"],
    "is": ["Flokkur"],
    "it": ["CAT", "Categoria"],
    "ja": ["カテゴリ"],
    "jbo": ["klesi"],
    "jv": ["Kategori"],
    "ka": ["კატეგორია"],
    "kaa": ["Sanat", "Kategoriya", "Санат", "سانات"],
    "kab": ["Taggayt"],
    "kbd": ["Категория", "Категориэ"],
    "kbp": ["Catégorie"],
    "kg": ["Kalasi"],
    "kk": ["Sanat", "Санат", "سانات"],
    "kl": ["Sumut_atassuseq", "Kategori", "Sumut atassuseq"],
    "km": ["ចំនាត់ថ្នាក់ក្រុម", "ចំណាត់ក្រុម", "ចំណាត់ថ្នាក់ក្រុម"],
    "kn": ["ವರ್ಗ"],
    "ko": ["분류"],
    "koi": ["Категория"],
    "krc": ["Категория"],
    "ks": ["زٲژ"],
    "ksh": ["Saachjropp", "Saachjrop", "Katejori", "Kategorie", "Saachjrupp", "Kattejori", "Sachjrop"],
    "ku": ["Kategorî", "پۆل"],
    "kv": ["Категория"],
    "kw": ["Class", "Klass"],
    "ky": ["Категория"],
    "la": ["Categoria"],
    "lad": ["Kateggoría", "Katēggoría", "Categoría"],
    "lb": ["Kategorie"],
    "lbe": ["Категория"],
    "lez": ["Категория"],
    "lfn": ["Categoria"],
    "li": ["Categorie", "Kategorie"],
    "lij": ["Categorîa", "Categoria"],
    "lmo": ["Categuria", "Categoria"],
    "ln": ["Catégorie"],
    "lo": ["ໝວດ"],
    "lrc": ["دأسە"],
    "lt": ["Kategorija"],
    "ltg": ["Kategoreja"],
    "lv": ["Kategorija"],
    "mai": ["CA", "श्रेणी"],
    "map-bms": ["Kategori"],
    "mdf": ["Категорие", "Категория"],
    "mg": ["Sokajy", "Catégorie"],
    "mhr": ["Категория", "Категорий"],
    "min": ["Kategori"],
    "mk": ["Категорија"],
    "ml": ["വിഭാഗം", "വി", "വർഗ്ഗം", "വ"],
    "mn": ["Ангилал"],
    "mr": ["वर्ग"],
    "mrj": ["Категори", "Категория"],
    "ms": ["Kategori"],
    "mt": ["Kategorija"],
    "mwl": ["Catadorie", "Categoria"],
    "my": ["ကဏ္ဍ"],
    "myv": ["Категория"],
    "mzn": ["رج", "رده"],
    "nah": ["Neneuhcāyōtl", "Categoría"],
    "nap": ["Categurìa", "Categoria"],
    "nds": ["Kategorie"],
    "nds-nl": ["Categorie", "Kattegerie", "Kategorie"],
    "ne": ["श्रेणी"],
    "new": ["पुचः"],
    "nl": ["Categorie"],
    "nn": ["Kategori"],
    "no": ["Kategori"],
    "nrm": ["Catégorie"],
    "nso": ["Setensele"],
    "nv": ["Tʼááłáhági_átʼéego", "Tʼááłáhági átʼéego"],
    "oc": ["Categoria"],
    "olo": ["Kategourii"],
    "or": ["ବିଭାଗ", "ଶ୍ରେଣୀ"],
    "os": ["Категори"],
    "pa": ["ਸ਼੍ਰੇਣੀ"],
    "pcd": ["Catégorie"],
    "pdc": ["Abdeeling", "Kategorie"],
    "pfl": ["Kadegorie", "Sachgrubb", "Kategorie"],
    "pi": ["विभाग"],
    "pl": ["Kategoria"],
    "pms": ["Categorìa"],
    "pnb": ["گٹھ"],
    "pnt": ["Κατηγορίαν"],
    "ps": ["وېشنيزه"],
    "pt": ["Categoria"],
    "qu": ["Katiguriya"],
    "rm": ["Categoria"],
    "rmy": ["Shopni"],
    "ro": ["Categorie"],
    "roa-rup": ["Categorie"],
    "roa-tara": ["Categoria"],
    "ru": ["Категория", "К"],
    "rue": ["Категория", "Катеґорія"],
    "rw": ["Ikiciro"],
    "sa": ["वर्गः"],
    "sah": ["Категория"],
    "sat": ["ᱛᱷᱚᱠ"],
    "sc": ["Categoria"],
    "scn": ["Catigurìa"],
    "sd": ["زمرو"],
    "se": ["Kategoriija"],
    "sg": ["Catégorie"],
    "sh": ["Kategorija", "Категорија"],
    "si": ["ප්‍රවර්ගය"],
    "sk": ["Kategória"],
    "sl": ["Kategorija"],
    "sq": ["Kategoria", "Kategori"],
    "sr": ["Kategorija", "Категорија"],
    "srn": ["Categorie", "Guru"],
    "stq": ["Kategorie"],
    "su": ["Kategori"],
    "sv": ["Kategori"],
    "sw": ["Jamii"],
    "szl": ["Kategoryjo", "Kategoria"],
    "ta": ["பகுப்பு"],
    "tcy": ["ವರ್ಗೊ"],
    "te": ["వర్గం"],
    "tet": ["Kategoría", "Kategoria"],
    "tg": ["Гурӯҳ"],
    "th": ["หมวดหมู่"],
    "ti": ["መደብ"],
    "tk": ["Kategoriýa"],
    "tl": ["Kategorya", "Kaurian"],
    "tpi": ["Grup"],
    "tr": ["Kategori", "KAT"],
    "tt": ["Төркем", "Törkem", "Категория"],
    "ty": ["Catégorie"],
    "tyv": ["Аңгылал", "Категория"],
    "udm": ["Категория"],
    "ug": ["تۈر"],
    "uk": ["Категория", "Категорія"],
    "ur": ["زمرہ"],
    "uz": ["Turkum", "Kategoriya"],
    "vec": ["Categoria"],
    "vep": ["Kategorii"],
    "vi": ["Thể_loại", "Thể loại"],
    "vls": ["Categorie"],
    "vo": ["Klad"],
    "wa": ["Categoreye"],
    "war": ["Kaarangay"],
    "wo": ["Wàll", "Catégorie"],
    "wuu": ["分类"],
    "xal": ["Янз", "Әәшл"],
    "xmf": ["კატეგორია"],
    "yi": ["קאטעגאריע", "קאַטעגאָריע"],
    "yo": ["Ẹ̀ka"],
    "za": ["分类"],
    "zea": ["Categorie"],
    "zh": ["分类", "分類", "CAT"],
    "zh-classical": ["分類", "CAT"],
    "zh-min-nan": ["分類", "Lūi-pia̍t"],
    "zh-yue": ["分类", "分類", "类", "類"],
}

In [7]:
import re
import mwparserfromhell

def parse_and_clean_wikitext(wikitext, lang='en'):
    """Strips formatting and unwanted sections from raw page content."""
    wikicode = mwparserfromhell.parse(wikitext)

    # Filters for magic words that are parser instructions -- e.g., __NOTOC__
    re_rm_magic = re.compile("__[A-Z]*__", flags=re.UNICODE)

    # Filters for file/image links.
    media_prefixes = '|'.join(['File', 'Image', 'Media'] + MEDIA_ALIASES.get(lang, []))
    re_rm_wikilink = re.compile(f"^(?:{media_prefixes}):", flags=re.IGNORECASE | re.UNICODE)

    def rm_wikilink(obj):
        return bool(re_rm_wikilink.match(str(obj.title)))

    # Filters for references and tables
    def rm_tag(obj):
        return str(obj.tag).lower() in {"ref", "table", "div", "gallery"}

    # Leave category links in-place but remove the category prefixes
    cat_prefixes = '|'.join(['Category'] + CAT_ALIASES.get(lang, []))
    re_clean_wikilink = re.compile(f"^(?:{cat_prefixes}):", flags=re.IGNORECASE | re.UNICODE)

    def is_category(obj):
        return bool(re_clean_wikilink.match(str(obj.title)))

    def clean_wikilink(obj):
        text = obj.__strip__()
        text = re.sub(re_clean_wikilink, '', text)
        obj.text = text

    def try_replace_obj(obj):
        try:
            clean_wikilink(obj)
        except ValueError:
            # For nested objects, they've sometimes already been removed.
            pass

    def try_remove_obj(obj, section):
        try:
            section.remove(obj)
        except ValueError:
            # For nested objects, they've sometimes already been removed.
            pass

    section_text = []
    # Filter individual sections to clean.
    for section in wikicode.get_sections(flat=True, include_lead=True, include_headings=True):
        for obj in section.ifilter_wikilinks(recursive=True):
            if rm_wikilink(obj):
                try_remove_obj(obj, section)
            elif is_category(obj):
                try_replace_obj(obj)
        for obj in section.ifilter_tags(matches=rm_tag, recursive=True):
            try_remove_obj(obj, section)

        section_text.append(re.sub(re_rm_magic, '', section.strip_code().strip()))
    #return " ".join(section_text)
    text = " ".join(section_text)
    return clean(text)

In [8]:
common_forbidden_patterns =  [
    "<!--.*?-->",  # this would remove any commented-out text, which is rare but sometimes substantial
    "\[\[category:.*?\]\]", # remove categories
    "{{.*?}}", # remove template text
    "&amp;",
    "&lt;",
    "&gt;",
    r"<ref[^<]*<\/ref>",  # remove reference tags
    "<[^>]*>",
    "\|left",
    "\|\d+px",
    r"(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b",
    "\|thumb",
    "\|right",
    "\[\[image:[^\[\]]*",
    "\[\[category:([^|\]]*)[^]]*\]\]",
    "\[\[[a-z\-]*:[^\]]*\]\]",
    "\[",
    "\]",
    "\{[^\}]*\}",
    r"\n",
    " +",
    r"[^a-zA-Z0-9 ]",
    r"\b[a-zA-Z]\b"
]

compiled_patterns = []
for pattern in common_forbidden_patterns:
    compiled_patterns.append(re.compile(pattern))

In [9]:
def clean(text):
    for p in compiled_patterns:       
        plaintext = re.sub(p, ' ', text)
        return ' '.join(plaintext.split())

In [10]:
#pip install mwparserfromhell

In [11]:
pages['cleaned_text'] = pages['text'].apply(lambda x: parse_and_clean_wikitext(x))

In [12]:
#pages['text'][5]

In [13]:
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))  
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
from string import digits
remove_digits = str.maketrans('', '', digits)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def preprocess(content):
    if isinstance(content, str):
        processed = content.translate(remove_digits)
    else:
        processed = content
    processed = processed.lower()
    processed = tokenizer.tokenize(processed)
    processed = [lemmatizer.lemmatize(token) for token in processed ]
    processed = [token for token in processed if token not in stop_words]

    return processed

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = preprocess)

In [14]:
from revBERT.keybert import KeyBERT
kw_model = KeyBERT()

In [15]:
def feature_extract(data):
    #keywords = kw_model.extract_keywords(data, top_n = 30, vectorizer = vectorizer)
    try:
        keywords = kw_model.extract_keywords(data, top_n = 20, vectorizer = vectorizer)
        #print(keywords)
        return str(keywords)
    except:
        return []

In [17]:
import warnings
warnings.filterwarnings("ignore")
pages['keywords'] = pages['cleaned_text'].apply(lambda x: feature_extract(x))

In [18]:
pages.rename(columns={'id':'page_id'},inplace=True)
pages.head()

,page_id,title,text,url,cleaned_text,keywords
0,5399,Colorado,{{short description|State of the United States...,https://en.wikipedia.org/wiki/Colorado,"Colorado (, other variants) is a state in the ...","[('colorado', 0.574), ('coloradoan', 0.441), (..."
1,5766,Clement Attlee,{{Redirect|Attlee}}\n{{short description|Prime...,https://en.wikipedia.org/wiki/Clement_Attlee,"Clement Richard Attlee, 1st Earl Attlee, (3 Ja...","[('churchill', 0.4173), ('thatcher', 0.3769), ..."
2,6749,Cheerleading,{{about|American-style cheerleading|Japanese-s...,https://en.wikipedia.org/wiki/Cheerleading,Cheerleading is an activity in which the parti...,"[('cheerleading', 0.6736), ('cheerleader', 0.5..."
3,11668,Follies,{{Other uses|Follies (disambiguation)}}\n{{Inf...,https://en.wikipedia.org/wiki/Follies,Follies is a musical with music and lyrics by ...,"[('broadway', 0.4292), ('broadwayfollies', 0.4..."
4,15080,Indian removal,{{Short description|Early 19th-century United ...,https://en.wikipedia.org/wiki/Indian_removal,Indian removal is the former United States gov...,"[('iroquois', 0.3307), ('cherokee', 0.313), ('..."


def only_keys(col):
    l = []
    for i in col.split('),'):
        #print(i.split(',')[0][2:])
        l.append(i.split(',')[0][2:])
    return l

In [36]:
import math
import re
def TI(row):
    d = {}
    #N = len(row)
    #key_lists = row.values
    for keys in row:
        #print(keys)
        key_split = keys.split('),')
        for key in key_split:
            token_val = key.split(',')
            #token = token_val[0].replace("_", " ")
            token = re.sub(r'[^\w\s|\s+|\s+$]','', token_val[0]).strip() #no space
            token = lemmatizer.lemmatize(token)
            val = re.sub(r'[^\-|\.|0-9]','', token_val[1]) #no number
            if token in d.keys():
                d[token]+= float(val)
                #d[token]['TI'] += float(val)
                #d[token]['DF'] += 1
            else: 
                d[token] = float(val)
                #d[token] = {}
                #d[token]['TI'] = float(val)
                #d[token]['DF'] = 1
            #d['token']['TI'] =  round(d['token']['TI'],3)
            #d['token']['DF'] = log(N/d['toke']['DF'])
                
    return d

In [20]:
def measure_TI(df):
    all_df = pd.DataFrame()
    all_df['keys'] = df.groupby('topic_id')['keywords'].apply(list)
#l = sample.groupby('item')['keywords'].apply(lambda x: IDF(x))
    all_df = all_df.reset_index()
    all_df['TI'] = all_df['keys'].apply(lambda x: TI(x))
    return all_df

In [33]:
topic_page = pd.merge(topic_info[['page_id', 'id', 'title']], pages[['page_id', 'title', 'keywords']], on ='page_id', how='inner')
topic_page.rename(columns={'title_x':'topic_title', 'title_y':'page_title', 'id':'topic_id'},inplace=True)
topic_page.head()

,page_id,topic_id,topic_title,page_title,keywords
0,45341506,1,Agriculture,Monsanto legal cases,"[('monsanto', 0.5933), ('gmos', 0.4758), ('bio..."
1,54101623,1,Agriculture,Cow vigilante violence in India,"[('cow', 0.4343), ('cattle', 0.4032), ('livest..."
2,31728658,1,Agriculture,Pesticide regulation in the United States,"[('pesticide', 0.4293), ('fda', 0.4109), ('reg..."
3,38121957,1,Agriculture,Census of agriculture,"[('census', 0.5091), ('agriculture', 0.4622), ..."
4,40838871,1,Agriculture,Supply management (Canada),"[('dairy', 0.4054), ('agrifood', 0.3538), ('ag..."


In [37]:
all_df = measure_TI(topic_page)

In [38]:
all_df.head(5)

,topic_id,keys,TI
0,1,"[[('monsanto', 0.5933), ('gmos', 0.4758), ('bi...","{'monsanto': 0.5933, 'gmos': 0.4758, 'biotechn..."
1,2,"[[('projector', 0.4261), ('imax', 0.4232), ('r...","{'projector': 0.4261, 'imax': 0.4232, 'resolut..."
2,3,"[[('cook', 0.3715), ('drafted', 0.3107), ('sto...","{'cook': 0.3715, 'drafted': 1.5066, 'stockton'..."
3,4,"[[('luftwaffe', 0.5141), ('bomber', 0.4788), (...","{'luftwaffe': 2.1009, 'bomber': 1.3408, 'aircr..."
4,5,"[[('mlb', 0.5039), ('inning', 0.4394), ('posts...","{'mlb': 4.288599999999999, 'inning': 2.9442999..."


import math
def TI_IDF(df, all_tokens):
    TIIDF = {}
    N = len(df['keys'])
    for keys in df['TI_DF']:
        all_tokens.add(keys)
        DF = df['TI_DF'][keys]['DF']
        TI = df['TI_DF'][keys]['TI']
        #IDF = math.log(N/(DF+1))
        #TIIDF[keys] = round(TI * IDF,3)
        TIIDF[keys] = round(TI ,3)
    return TIIDF

all_tokens = set()
all_df['TI_IDF'] = all_df.apply(lambda x: TI_IDF(x, all_tokens), axis=1)
all_df.head()

In [39]:
def top_keywords(row):
    return [k for k, v in sorted(row.items(), key=lambda item: item[1])][::-1]

In [64]:
all_df['top_keys'] = all_df['TI'].apply(lambda x: top_keywords(x))
topic_keys = all_df[['topic_id', 'top_keys']]
topic_keys

,topic_id,top_keys
0,1,"[agricultural, agriculture, pesticide, livesto..."
1,2,"[sydney, macquarie, nsw, architecture, parrama..."
2,3,"[quarterback, nfl, drafted, athletics, heisman..."
3,4,"[aircraft, aviation, flown, luftwaffe, airline..."
4,5,"[mlb, mets, sox, baseball, phillies, astros, p..."
5,6,"[academic, alumnus, faculty, harvard, nobel, j..."
6,7,"[coached, coach, winger, midfielder, dean, wre..."
7,8,"[actor, cast, filmmaker, daisuke, voiced, yama..."
8,9,"[boxer, boxing, welterweight, heavyweight, fig..."
9,10,"[buddhism, buddhist, dharma, buddha, sutta, su..."


In [61]:
df = topic_page.drop_duplicates(subset=['topic_id', 'topic_title'], keep='first')

In [65]:
#topic_page
topic_keys = pd.merge(topic_keys, df[['topic_id', 'topic_title']], how='inner', on='topic_id')
topic_keys

,topic_id,top_keys,topic_title
0,1,"[agricultural, agriculture, pesticide, livesto...",Agriculture
1,2,"[sydney, macquarie, nsw, architecture, parrama...",Architecture
2,3,"[quarterback, nfl, drafted, athletics, heisman...",Athletics
3,4,"[aircraft, aviation, flown, luftwaffe, airline...",Aviation
4,5,"[mlb, mets, sox, baseball, phillies, astros, p...",Baseball
5,6,"[academic, alumnus, faculty, harvard, nobel, j...",Biography/science and academia work group
6,7,"[coached, coach, winger, midfielder, dean, wre...",Biography/sports and games work group
7,8,"[actor, cast, filmmaker, daisuke, voiced, yama...",Biography/WikiProject Actors and Filmmakers
8,9,"[boxer, boxing, welterweight, heavyweight, fig...",Boxing
9,10,"[buddhism, buddhist, dharma, buddha, sutta, su...",Buddhism


In [55]:
topic_keys['topic_title'] = title #topics['title']

In [42]:
topic_keys.to_parquet('data/top_keys.parquet')

In [43]:
#pip install xlsxwriter

In [44]:
topic_keys.to_excel("data/top_keys.xlsx", engine='xlsxwriter')

In [56]:
topic_keys

,topic_id,top_keys,topic_title
0,1,"[agricultural, agriculture, pesticide, livesto...",Agriculture
1,2,"[sydney, macquarie, nsw, architecture, parrama...",Architecture
2,3,"[quarterback, nfl, drafted, athletics, heisman...",Engineering
3,4,"[aircraft, aviation, flown, luftwaffe, airline...",Athletics
4,5,"[mlb, mets, sox, baseball, phillies, astros, p...",Running
5,6,"[academic, alumnus, faculty, harvard, nobel, j...",Aviation
6,7,"[coached, coach, winger, midfielder, dean, wre...",Baseball
7,8,"[actor, cast, filmmaker, daisuke, voiced, yama...",Biography/science and academia work group
8,9,"[boxer, boxing, welterweight, heavyweight, fig...",Jewish history
9,10,"[buddhism, buddhist, dharma, buddha, sutta, su...",Journalism
